<a href="https://colab.research.google.com/github/xandreiAThome/machine-translation-nlp1k/blob/main/nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Machine Translation

## Preprocess

Load the aligned verses from the tsv, clean the string from any non alphabetic characters. Remove any verses that have no verse for either of the two language, and use the class from the datasets library to structure the data and be ready for training.

In [4]:
import regex as re

def clean_string(input_string):
    cleaned = re.sub(r"[^\p{L}\s]", "", input_string.strip().lower())
    return cleaned

def process(example):
    src = example["src"].strip()
    tgt = example["tgt"].strip()

    # skip invalid pairs
    if src.lower() == "<no verse>" or tgt.lower() == "<no verse>":
        return {"src": None, "tgt": None}

    return {
        "src": clean_string(src),
        "tgt": clean_string(tgt),
    }

In [5]:
src_lang = "Bikolano"
target_lang = "Tagalog"

In [6]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files="data/dataset/Bikolano_Tagalog_Parallel.tsv",
    delimiter="\t",
)

dataset = dataset["train"].select_columns([src_lang, target_lang])
dataset = dataset.rename_columns({src_lang: "src", target_lang: "tgt"})

# Get initial dataset length
initial_dataset_length = len(dataset)

dataset = dataset.map(process)

# remove rows with None (invalid)
dataset = dataset.filter(lambda x: x["src"] is not None and x["tgt"] is not None)

# Calculate skipped verses
skipped = initial_dataset_length - len(dataset)
print(f"skipped verses: {skipped}")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/33900 [00:00<?, ? examples/s]

Filter:   0%|          | 0/33900 [00:00<?, ? examples/s]

skipped verses: 27


Lets look at the first 5 aligned verses

In [7]:
display(dataset[:5])

{'src': ['si adan iyo an ama ni set asin si set iyo an ama ni enos na ama ni kenan',
  'si kenan iyo an ama ni mahalalel na ama ni jared',
  'si jared iyo an ama ni enoc na ama ni metusela si metusela iyo an ama ni lamec',
  'na iyo an ama ni noe si noe nagkaigwa nin tolong aking lalaki na iyo si sem ham asin si jafet',
  'an mga aking lalaki ni jafet iyo si gomer magog madai javan tubal mesec asin tiras'],
 'tgt': ['sina adan set enos',
  'kenan mahalalel jared',
  'enoc matusalem lamec',
  'noe sem ham at jafet',
  'ang mga anak ni jafet ay sina gomer magog madai javan tubal meshec at tiras']}

## Setting up Trainer
We will use facebook's No Language Left Behind Model as the base model to fine tune using our dataset. It is performant even on low resource languages thats why our group decided to use it.

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [9]:
def tokenize(batch):
    model_inputs = tokenizer(batch["src"], truncation=True, max_length=128)
    labels = tokenizer(batch["tgt"], truncation=True, max_length=128).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/33873 [00:00<?, ? examples/s]

Let us split the training data to also have a dataset for evaluation after training.

In [10]:
split = tokenized_dataset.train_test_split(test_size=0.1)
train_data = split["train"]
eval_data = split["test"]

In [13]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./nllb-bcl-tgl",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=8,
    eval_strategy="epoch", # Changed from evaluate_during_training
    save_strategy="epoch",
    logging_steps=50,
    fp16=True,
    gradient_accumulation_steps=2,  # effective batch size = 8
    weight_decay=0.01,
    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

NameError: name 'Seq2SeqTrainer' is not defined

In [ ]:
trainer.train()
trainer.save_model("./nllb-bcl-tgl")
tokenizer.save_pretrained("./nllb-bcl-tgl")
